# Demonstration of mixed-unit partition allocation

The `allocation_construct` module closely follows the notation and structure of the following article

> Majeau-Bettez, G., Wood, R. & Strømman, A.H., 2014. Unified Theory of Allocations and Constructs in Life Cycle Assessment and Input-Output Analysis. _Journal of Industrial Ecology_, 18(5), pp.747–770. http://doi.wiley.com/10.1111/jiec.12142

and its [supporting information](http://onlinelibrary.wiley.com/store/10.1111/jiec.12142/asset/supinfo/jiec12142-sup-0001-SupMatS1.pdf?v=1&s=9417c703c400a9eb8994b249289346bf276713ae)
presents many simple examples and demonstrations.



However, it does _not_ illustrate a situation where one industry is partitionned following a certain property (e.g., portionately to mass) and another industry is partitionned proportionately to some other property (e.g., energy content). Presenting such a situation is the purpose of this demo.

### Defining the Supply and Use system with multifunctional activities

In [1]:
# Basic imports
import sys
import numpy as np
import pandas as pd
sys.path.append('/home/bill/software/allocation_construct/')
import allocation_construct as ac

In [2]:
# Define labels
l_ind = np.array(['I', 'J', 'K'])
l_com = np.array(['i (kg)', 
                  'j (kg)', 
                  'k (kg)'])
l_ext = np.array(['emission (kg)', 'ressource (kg)'])

# l_out: commodities traceable to their producing industries (j produced by I, k produced by K, etc.)
l_out = []
for i in l_ind:
    for j in l_com:
        l_out.append(i + '_' + j)
l_out = np.array(l_out)

In [3]:
# Supply table (what each industry produces)
# Note coproduction in industries I and J
                 # I   J    K
V = pd.DataFrame([[2., 0.3, 0.],  # i
                  [1., 1.,  0.],  # j
                  [0., 0., 11.]], # k
                 index=l_com,
                 columns=l_ind)
V

,I,J,K
i (kg),2.0,0.3,0.0
j (kg),1.0,1.0,0.0
k (kg),0.0,0.0,11.0


In [4]:
# untraceable Use table
# What products each industry uses

Uu = pd.DataFrame(
        data= [[ 0.  ,  0.  ,  0.  ],
               [ 0.  ,  0.  ,  0.75],
               [ 4.  ,  2   ,  0.  ]],
        index=l_com, columns=l_ind)
Uu

,I,J,K
i (kg),0.0,0.0,0.00
j (kg),0.0,0.0,0.75
k (kg),4.0,2.0,0.00


In [5]:
# Exchanges with the environment
# Note negative flows for inputs

G = pd.DataFrame([[1.0,    0.7,    0],
                  [0,      0,    -10.25]],
                index=l_ext, columns=l_ind)
G

,I,J,K
emission (kg),1.0,0.7,0.00
ressource (kg),0.0,0.0,-10.25


In [6]:
# For the sake of completeness, final demand by households for each product
y = pd.Series([2.3, 1.25, 5], l_com)
y

i (kg)    2.30
j (kg)    1.25
k (kg)    5.00
dtype: float64

### Selecting properties that will guide partition allocation (PA)

The properties that guide the definition of partition coefficients are defined in the commodity-by-industry matrix $ \Psi $. 

* Let us have the requirements of industries _I_ and _K_ allocated proportionately to mass. Consequently, the columns _I_ and _K_ in $\Psi$ will hold the mass per unit of each commodity (in this case, always 1.0 kg).

* Then, let us have industry _J_ allocated proportionately to the economic value of its coproducts. To do that, let us put the value of each product in column _J_ of $\Psi$.

In [7]:
value_i = 0.5 # $/kg
value_j = 0.75 # $/kg
value_k = 0.89 # $/kg

mass_i = 1.0 # kg/kg
mass_j = 1.0 # kg/kg
mass_k = 1.0 # kg/kg

PSI = pd.DataFrame(
    data = [[mass_i, value_i, mass_i],
            [mass_j, value_j, mass_j], 
            [mass_k, value_k, mass_k]],
    index=l_com, columns=l_ind)

PSI

,I,J,K
i (kg),1.0,0.50,1.0
j (kg),1.0,0.75,1.0
k (kg),1.0,0.89,1.0


### Performing the allocation

In [8]:
Z, A, nn_in, nn_out, G_all, F = ac.pa(Uu.values, V.values, PSI.values, G=G.values)

### Format the matrices

In [9]:
# The normalized, allocated requirements 
# for the production of each commodity in each industry
A = pd.DataFrame(A, l_com[nn_in], l_out[nn_out])
A

,I_i (kg),I_j (kg),J_i (kg),J_j (kg),K_k (kg)
i (kg),0.000000,0.000000,0.000000,0.000000,0.000000
j (kg),0.000000,0.000000,0.000000,0.000000,0.068182
k (kg),1.333333,1.333333,1.111111,1.666667,0.000000


In [10]:
# The normalized, allocated exchanges with the environment 
# for the production of each commodity in each industry
F = pd.DataFrame(F, l_ext, l_out[nn_out])
F

,I_i (kg),I_j (kg),J_i (kg),J_j (kg),K_k (kg)
emission (kg),0.333333,0.333333,0.388889,0.583333,0.000000
ressource (kg),0.000000,0.000000,0.000000,0.000000,-0.931818
